# Fasttext

Скачаем это все сюда, потому что в оперативку моего ноута оно не влезло.

## Подготовка к запуску в колабе

In [0]:
!wget 'http://vectors.nlpl.eu/repository/11/181.zip'

--2019-10-02 19:04:02--  http://vectors.nlpl.eu/repository/11/181.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2622716217 (2.4G) [application/zip]
Saving to: ‘181.zip’

181.zip             100%[===================>]   2.44G  67.4MB/s    in 49s     

2019-10-02 19:04:51 (50.9 MB/s) - ‘181.zip’ saved [2622716217/2622716217]



In [0]:
!unzip '181.zip' -d 'fasttext'

Archive:  181.zip
  inflating: /content/fasttext/meta.json  
  inflating: /content/fasttext/model.model  
  inflating: /content/fasttext/model.model.vectors_ngrams.npy  
  inflating: /content/fasttext/model.model.vectors.npy  
  inflating: /content/fasttext/model.model.vectors_vocab.npy  
  inflating: /content/fasttext/README  


In [0]:
from gensim.models.keyedvectors import KeyedVectors

fast_model = '/content/fasttext/model.model'

In [0]:
model = KeyedVectors.load(fast_model)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Препроцессинг

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 7.1MB 13.6MB/s 


In [0]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [0]:
preprocessing('Я так больше не могу', del_stopwords=False)

['я', 'так', 'большой', 'не', 'мочь']

## Получение вектора документа из модели

Проверка на наличие в модели не работает: падает с ошибкой AttributeError, поэтому будем обрабатывать исключение.

In [0]:
wv = model.wv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [0]:
import numpy as np

def lookup(doc, wv):
    """
    Checks if all words in model and returns a vector

    :param doc: text string
    :param wv: model.wv, WordVectors
    :return: vector of the document
    """
    d = preprocessing(doc, del_stopwords=False)
    checked = []

    for word in d:
        try:
            word in wv
        except AttributeError:
            continue
        checked.append(wv[word])

    vec = np.mean(checked, axis=0)
    return vec 


def cos_sim(v1, v2):
    """Counts cosine similarity between two vectors"""
    return np.inner(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))



def get_dist(text1, text2, wv):
    """
    Counts distanse between two documents
    :param text1: str
    :param text2: str
    :param wv: model.wv, WordVectors

    """
    t1 = lookup(text1, wv)
    t2 = lookup(text2, wv)
    dist = cos_sim(t1, t2)
    return dist


Пример работы

In [0]:
t1 = 'Я ничего не успеваю'
t2 = 'Ничего не будет хорошо'
get_dist(t1, t2, wv)

0.793653

## Индексирование корпуса

Дальше я загрузила очищенный корпус Quora question pairs из предыдущей домашки.

In [0]:
corpus = 'quora_question_pairs_rus.csv'

In [0]:
with open(corpus, 'r', encoding='utf-8') as f:
    print(f.read(300))

,question1,question2,is_duplicate
0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украдет кохинор кох-и-ноор-алмаз назад",0
1,"как я могу увеличить скорость моего интернет-соединения, используя vpn",как повысить скорость интернета путем взлома через dns,0
2,"п


In [0]:
import csv

def get_data(corpus, wv, stop=5000):
    """
    :param corpus: path to csv file with corpus
    :param wv: model.KeyedVectors.wv, WordVectors
    :param stop: int, how many lines we want to get

    :return: 
        indexed -> list of document vectors
        id_to_text -> dict, map of text_id to raw text. 
        query_to_dupl -> dict, query:id of its duplicate

    """
    indexed = []
    id_to_text = {}
    query_to_dupl_id = {}
    counter = 0

    with open(corpus, 'r', encoding='utf-8') as f:
        r = csv.reader(f)
        for line in r:

            if line[0] == '':
                continue

            _id, text, query, isduplicate = line
            id_to_text[_id] = text

            if isduplicate == '1':
                query_to_dupl_id[query] = _id
                
            indexed.append(lookup(text, wv))
                
            counter += 1
            if counter >= stop:
                break
    return indexed, id_to_text, query_to_dupl_id


In [0]:
%%time
indexed, id_to_text, query_to_dupl_id = get_data(corpus, wv, stop=50000)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 2min 5s, sys: 1.85 s, total: 2min 6s
Wall time: 2min 7s


In [0]:
import pickle
with open("Indexed_FT.pickle", 'wb') as f:
    pickle.dump((indexed, id_to_text, query_to_dupl_id), f)

In [0]:
with open("Indexed_FT.pickle", 'rb') as f:
    indexed, id_to_text, query_to_dupl_id = pickle.load(f)

## Поиск в корпусе


In [0]:
def search(query, wv, indexed):
    """
    Search tool

    :param query: str
    :param wv: model.KeyedVectors.wv
    :param indexed: iterable of vectors
    """

    query_v = lookup(query, wv)
    
    result = {}
    for i, doc_vector in enumerate(indexed):
        score =  cos_sim(query_v, doc_vector)
        if type(score) is np.float32:
            result[i] = score
    
    return sorted(result.items(), key=lambda x: x[1], reverse=True)


In [0]:
%%time
search('Не хочу быть лингвистом, хочу быть хорошим геологом', wv, indexed)[:5]

CPU times: user 592 ms, sys: 3.99 ms, total: 596 ms
Wall time: 601 ms


[(6, 0.86064434),
 (14150, 0.8143773),
 (34884, 0.7928043),
 (21972, 0.79240125),
 (34574, 0.7921281)]

In [0]:
id_to_text['6']

'как я могу быть хорошим геологом?'

## Оценка качества поиска

In [0]:
def get_score(indexed, query_to_dupl_id, wv, test=100):
    """
    Counts the quality of the search (from 0 to 1.0)
    """
    test_query = list(query_to_dupl_id.keys())

    if test != 0:
        test_query =  test_query[:test]
    
    test_len = len(test_query)
    counter = 0

    for q in test_query:
        dupl_id = int(query_to_dupl_id[q])

        results = search(q, wv, indexed)[:5]
        text_ids = [result[0] for result in results]

        if dupl_id in text_ids:
            counter += 1
    
    return counter / test_len


### Оценка качества поиска

In [0]:
%%time
get_score(indexed, query_to_dupl_id, wv)

CPU times: user 58.1 s, sys: 35.9 ms, total: 58.2 s
Wall time: 58.3 s


0.42

Не то чтобы супер (а на 5000 в корпусе около 70% было т.т).

# ELMo

Скачаем модель ELMo и распакуем ее.

In [1]:
!wget "http://vectors.nlpl.eu/repository/11/196.zip"

--2019-10-05 07:31:12--  http://vectors.nlpl.eu/repository/11/196.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206986345 (197M) [application/zip]
Saving to: ‘196.zip’

196.zip             100%[===================>] 197.40M  23.1MB/s    in 13s     

2019-10-05 07:31:26 (14.7 MB/s) - ‘196.zip’ saved [206986345/206986345]



In [2]:
!unzip '196.zip' -d 'ELMO'

Archive:  196.zip
  inflating: /content/ELMO/meta.json  
  inflating: /content/ELMO/model.hdf5  
  inflating: /content/ELMO/options.json  
  inflating: /content/ELMO/README    
  inflating: /content/ELMO/vocab.txt  


In [0]:
import os
os.mkdir('bilm')

## ELMo helpers from rep

В моей домашке они немного подправлены: изменено расширение файла со словарем, чтобы соответствовать модели

In [0]:
%load_ext autoreload

import time
import numpy as np
import tensorflow as tf
from elmo_helpers import tokenize, get_elmo_vectors, load_elmo_embeddings

tf.reset_default_graph()
elmo_path = 'ELMO'


## Загрузка модели

In [6]:
batcher, sentence_character_ids, elmo_sentence_input = load_elmo_embeddings(elmo_path)





Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





## Предобработка корпуса

In [0]:
corpus = 'quora_question_pairs_rus.csv'

In [0]:
import csv

def get_data_elmo(corpus, stop=5000):
    """
    Проходит по корпусу и токенизирует тексты.

    :param corpus: path to csv file with corpus
    :param stop: int, how many lines we want to get
    :return: 
        indexed -> list of list of strings
        id_to_text -> dict, map of text_id to raw text. 
        query_to_dupl -> dict, query:id of its duplicate

    """
    indexed = []
    id_to_text = {}
    query_to_dupl_id = {}
    counter = 0

    with open(corpus, 'r', encoding='utf-8') as f:
        r = csv.reader(f)
        for line in r:

            if line[0] == '':
                continue

            _id, text, query, isduplicate = line
            id_to_text[_id] = text

            if isduplicate == '1':
                query_to_dupl_id[query] = _id
                
            indexed.append(tokenize(text))
                
            counter += 1
            if counter >= stop:
                break
    return indexed, id_to_text, query_to_dupl_id

In [0]:
cleaned, id_to_text, query_to_dupl_id = get_data_elmo(corpus, stop=50000)

In [0]:
def crop_vec(vect, sent):
    """
    Crops dummy values

    :param vect: np.array, vector from ELMo
    :param sent: list of str, tokenized sentence
    :return: np.array

    """
    cropped_vector = vect[:len(sent), :]
    cropped_vector = np.mean(cropped_vector, axis=0)
    return cropped_vector

In [0]:
def indexing(cleaned, batcher, sentence_character_ids, elmo_sentence_input):
    """ 
    Indexing corpus
    :param cleaned: list if lists of str, tokenized documents from the corpus
    :param batcher, sentence_character_ids, elmo_sentence_input: ELMo model

    :return: matrix of document vectors
    """
    with tf.Session() as sess:
        # It is necessary to initialize variables once before running inference.
        sess.run(tf.global_variables_initializer())
        indexed = []
        for i in range(200, len(cleaned)+1, 200):
            sentences = cleaned[i-200 : i]
            elmo_vectors = get_elmo_vectors(
                sess, sentences, batcher, sentence_character_ids, elmo_sentence_input)

            for vect, sent in zip(elmo_vectors, sentences):
                cropped_vector = crop_vec(vect, sent)
                indexed.append(cropped_vector)
    return indexed

Индексируем корпус

In [86]:
from time import time
start = time()
indexed = indexing(cleaned, batcher, sentence_character_ids, elmo_sentence_input)
print('Затрачено секунд: ', time() - start)

Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 200
Sentences in this batch: 2

Затрачено секунд:  5569.673327922821


Сохраним индексированный корпус

In [0]:
import pickle

with open('Indexed_ELMO.pickle', 'wb') as f:
    pickle.dump((indexed, id_to_text, query_to_dupl_id), f)

## Поиск

In [0]:
# Эта функция есть выше в разделе fasttext, но продублируем на всякий случай
def cos_sim(v1, v2):
    """Counts cosine similarity between two vectors"""
    return np.inner(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [0]:
# Loading indexed corpus
import pickle

with open('Indexed_ELMO.pickle', 'rb') as f:
    indexed, id_to_text, query_to_dupl_id = pickle.load(f)

In [0]:
def prepare_query(query, batcher, sentence_character_ids, elmo_sentence_input):
    """ 
    Gets vector of query

    :param query: str
    :param batcher, sentence_character_ids, elmo_sentence_input: ELMo model
    
    :return: vector of query
    """
    q = [tokenize(query)]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        vector = crop_vec(get_elmo_vectors(sess, q, batcher,
                                           sentence_character_ids,
                                           elmo_sentence_input)[0], q[0])
    return vector

In [0]:
def search_tool_elmo(query, batcher, sentence_character_ids,
                     elmo_sentence_input, indexed):
    """
    Search query in corpus

    :param: query: str
    :param batcher, sentence_character_ids, elmo_sentence_input: ELMo model
    :param indexed: np.array, matrix of indexed corpus

    :return: list, sorted results
    """
    q = prepare_query(query, batcher, sentence_character_ids, 
                      elmo_sentence_input)

    result = {}
    for i, doc_vector in enumerate(indexed):
        score =  cos_sim(q, doc_vector)
        if type(score) is np.float32:
            result[i] = score
    
    return sorted(result.items(), key=lambda x: x[1], reverse=True)
    

In [0]:
Пример поиска

In [80]:
res = search_tool_elmo('Что нужно, чтобы стать хорошим геологом?', batcher,
                       sentence_character_ids, elmo_sentence_input, indexed)
print(res[:5])

Sentences in this batch: 1


[(6, 0.7573212), (500, 0.7192065), (427, 0.6865775), (363, 0.6859282), (544, 0.6816523)]


In [75]:
id_to_text['6']

'как я могу быть хорошим геологом?'

## Оценка качества поиска

In [0]:
def get_score_elmo(indexed, query_to_dupl_id, batcher, sentence_character_ids, 
                   elmo_sentence_input, test=100):
    """
    Counts the quality of the search (from 0 to 1.0)
    """
    test_query = list(query_to_dupl_id.keys())

    if test != 0:
        test_query =  test_query[:test]
    
    test_len = len(test_query)
    counter = 0

    for q in test_query:
        dupl_id = int(query_to_dupl_id[q])

        results = search_tool_elmo(q, batcher, sentence_character_ids,
                                   elmo_sentence_input, indexed)[:5]
        text_ids = [result[0] for result in results]

        if dupl_id in text_ids:
            counter += 1
    
    return counter / test_len

In [90]:
start = time()
print(get_score_elmo(indexed,query_to_dupl_id, batcher, sentence_character_ids, 
                   elmo_sentence_input))
print('Затрачено времени', time() - start)

Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
S

0.38
Затрачено времени 290.902761220932


М-да. Лучше не стало.

Но на небольших корпусах дает результаты лучше. Возможно, это какие-то ограничения моей функции для подсчета результатов :)